# Load DATA

In [ ]:
!pip install pyqt5 pandas pycartool

Load the exemple dataset

In [1]:
import os
import mne

import pandas as pd
import pycartool
import numpy as np

folder = fr"C:\Users\ferat\Downloads\EEG Class - Geneva - 2022\EEG Class - Geneva - 2022\1 Preprocessing\Data\4 ICA"
file = "corrado1_VEPcheck_20170428_112415_FilterDown"
vhdr_file = file + ".vhdr"
mkr_file = file + ".eeg.mrk"
bads_file = "corrado.Bad tracks.2.txt"

# data
raw = mne.io.read_raw(os.path.join(folder, vhdr_file))

# markers
df = pd.read_csv(os.path.join(folder, mkr_file), sep="\t", skiprows=1, names=['start', 'stop', 'label'])
df['start_time'] = df['start'] / raw.info['sfreq']
df['stop_time'] = df['stop'] / raw.info['sfreq']
df['duration'] = df['stop_time'] - df['start_time']
annotations = mne.Annotations(df['start_time'], df['duration'], df['label'])

# bads
with open(os.path.join(folder, bads_file), "r") as file:
    # Writing data to a file
    bads = file.readline()
bads = bads.split()

# positions
xyz_filename = os.path.join(folder, "..", "EGI 257.Geneva Average 13.10-10.xyz")
montage = pycartool.io.read_xyz(xyz_filename)

    
raw.set_annotations(annotations) 
raw.info['bads'] = bads
raw.set_montage(montage)

Extracting parameters from C:\Users\ferat\Downloads\EEG Class - Geneva - 2022\EEG Class - Geneva - 2022\1 Preprocessing\Data\4 ICA\corrado1_VEPcheck_20170428_112415_FilterDown.vhdr...
Setting channel info structure...


<RawBrainVision | corrado1_VEPcheck_20170428_112415_FilterDown.eeg, 257 x 137709 (550.8 s), ~359 kB, data not loaded>

In [6]:
%matplotlib qt
mne.viz.set_browser_backend('qt')  
raw.plot()

Using qt as 2D backend.


## ICA

ICA is [`mne.preprocessing.ica`](https://mne.tools/stable/generated/mne.preprocessing.ICA.html?highlight=ica#mne.preprocessing.ICA)

In [2]:
from mne.preprocessing import ICA

ICA?

### Method

`fastica`

Aapo Hyvärinen. Fast and robust fixed-point algorithms for independent component analysis. IEEE Transactions on Neural Networks, 10(3):626–634, 1999. doi:10.1109/72.761722.

`infomax`

Anthony J. Bell and Terrence J. Sejnowski. An information-maximization approach to blind separation and blind deconvolution. Neural Computation, 7(6):1129–1159, 1995. doi:10.1162/neco.1995.7.6.1129.

`infomax` + `fit_params=dict(extended=True)`

Te-Won Lee, Mark Girolami, and Terrence J. Sejnowski. Independent component analysis using an extended infomax algorithm for mixed subgaussian and supergaussian sources. Neural Computation, 11(2):417–441, 1999. doi:10.1162/089976699300016719.

`picard`

Pierre Ablin, Jean-Francois Cardoso, and Alexandre Gramfort. Faster Independent Component Analysis by preconditioning with hessian approximations. IEEE Transactions on Signal Processing, 66(15):4040–4049, 2018. doi:10.1109/TSP.2018.2844203.

In [3]:
ica = ICA(n_components=0.99,
          noise_cov=None,
          random_state=42,
          method='infomax', # picard solver
          fit_params=dict(extended=True), # extended infomax
          max_iter='auto',
          allow_ref_meg=False, # MEG only
          verbose=None)

### Fit

In [ ]:
ica.fit?

In [4]:
ica.fit(raw, decim=100)

Fitting ICA to data using 231 channels (please be patient, this may take a while)
Omitting 18165 of 137709 (13.19%) samples, retaining 119544 (86.81%) samples.


C:\Users\ferat\AppData\Local\Temp\ipykernel_16352\245588204.py:1: RuntimeWarning: The data has not been high-pass filtered. For good ICA performance, it should be high-pass filtered (e.g., with a 1.0 Hz lower bound) before fitting ICA.
  ica.fit(raw, decim=100)


Selecting by explained variance: 62 components
Computing Extended Infomax ICA
Fitting ICA took 10.0s.


Method,infomax
Fit,500 iterations on raw data (1196 samples)
ICA components,62
Explained variance,99.0 %
Available PCA components,231
Channel types,eeg
ICA components marked for exclusion,—


### Selecting Bad components

In [8]:
ica.plot_components(inst=raw, sphere=100, topomap_args={'extrapolate': 'local'})

[<MNEFigure size 1218.75x1055 with 20 Axes>,
 <MNEFigure size 1218.75x1055 with 20 Axes>,
 <MNEFigure size 1218.75x1055 with 20 Axes>,
 <MNEFigure size 487.5x325 with 2 Axes>]

    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
223 matching events found
No baseline correction applied
0 projection items activated


C:\Users\ferat\Miniconda3\envs\eeg39\lib\site-packages\mne\viz\epochs.py:543: MatplotlibDeprecationWarning: The join function was deprecated in Matplotlib 3.6 and will be removed two minor releases later.
  ax['evoked'].get_shared_x_axes().join(ax['evoked'], ax_im)


    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
223 matching events found
No baseline correction applied
0 projection items activated


C:\Users\ferat\Miniconda3\envs\eeg39\lib\site-packages\mne\viz\epochs.py:543: MatplotlibDeprecationWarning: The join function was deprecated in Matplotlib 3.6 and will be removed two minor releases later.
  ax['evoked'].get_shared_x_axes().join(ax['evoked'], ax_im)


...

C:\Users\ferat\Miniconda3\envs\eeg39\lib\site-packages\mne\viz\epochs.py:543: MatplotlibDeprecationWarning: The join function was deprecated in Matplotlib 3.6 and will be removed two minor releases later.
  ax['evoked'].get_shared_x_axes().join(ax['evoked'], ax_im)


    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
223 matching events found
No baseline correction applied
0 projection items activated


C:\Users\ferat\Miniconda3\envs\eeg39\lib\site-packages\mne\viz\epochs.py:543: MatplotlibDeprecationWarning: The join function was deprecated in Matplotlib 3.6 and will be removed two minor releases later.
  ax['evoked'].get_shared_x_axes().join(ax['evoked'], ax_im)


    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
223 matching events found
No baseline correction applied
0 projection items activated


C:\Users\ferat\Miniconda3\envs\eeg39\lib\site-packages\mne\viz\epochs.py:543: MatplotlibDeprecationWarning: The join function was deprecated in Matplotlib 3.6 and will be removed two minor releases later.
  ax['evoked'].get_shared_x_axes().join(ax['evoked'], ax_im)


In [ ]:
ica.plot_sources(inst=raw);

### MNE ICA labels

Li et al., (2022). MNE-ICALabel: Automatically annotating ICA components with ICLabel in Python. Journal of Open Source Software, 7(76), 4484, https://doi.org/10.21105/joss.04484


In [ ]:
!pip install mne-icalabel

In [9]:
from mne_icalabel import label_components

# assuming you have a Raw and ICA instance previously fitted
label_components(raw, ica, method='iclabel')

C:\Users\ferat\Miniconda3\envs\eeg39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\ferat\AppData\Local\Temp\ipykernel_16352\3350189304.py:4: RuntimeWarning: The provided Raw instance does not seem to be referenced to a common average reference (CAR). ICLabel was designed to classify features extracted from an EEG dataset referenced to a CAR (see the 'set_eeg_reference()' method for Raw and Epochs instances).
  label_components(raw, ica, method='iclabel')
C:\Users\ferat\AppData\Local\Temp\ipykernel_16352\3350189304.py:4: RuntimeWarning: The provided Raw instance is not filtered between 1 and 100 Hz. ICLabel was designed to classify features extracted from an EEG dataset bandpass filtered between 1 and 100 Hz (see the 'filter()' method for Raw and Epochs instances).
  label_components(raw, ica, method='ic

{'y_pred_proba': array([0.6666677 , 0.9886285 , 0.40531597, 0.57149345, 0.59760964,
        0.3962925 , 0.8080262 , 0.55837977, 0.7279339 , 0.77490735,
        0.916679  , 0.80369115, 0.50412714, 0.94437075, 0.8686347 ,
        0.8005514 , 0.81108254, 0.9745213 , 0.94322264, 0.80952203,
        0.8561602 , 0.7507868 , 0.7395763 , 0.91780955, 0.768227  ,
        0.9680057 , 0.6397254 , 0.95087075, 0.89622086, 0.6469287 ,
        0.5627378 , 0.8733658 , 0.87206966, 0.8304574 , 0.8041336 ,
        0.81005764, 0.8677387 , 0.94117725, 0.97572064, 0.9214253 ,
        0.60705495, 0.9737159 , 0.8963469 , 0.7008535 , 0.9833041 ,
        0.8833132 , 0.88100374, 0.48449558, 0.56015116, 0.6975404 ,
        0.9547759 , 0.8959241 , 0.8442444 , 0.54462004, 0.9934561 ,
        0.8356608 , 0.83280534, 0.8027995 , 0.6907849 , 0.33180505,
        0.57271194, 0.76940143], dtype=float32),
 'labels': ['brain',
  'brain',
  'brain',
  'other',
  'brain',
  'brain',
  'brain',
  'other',
  'other',
  'other',

### Apply

In [ ]:
raw_ica = ica.apply(raw)

In [ ]:
raw_ica.plot()

### Additional Preprocessing steps

In [ ]:
raw_ica.interpolate_bads()

In [ ]:
raw_ica.set_eeg_reference('average')

### Save

#### ICA

In [ ]:
ica.save("my_ica-ica.fif")

#### EEG

In [ ]:
mne.export.export_raw('raw_ica.vhdr', raw)